### Modelling a multi - echelon, multi - mode, multi - product, multi - period Supply Chain Transportation Problem considering stakeholder behaviourial tendencies 

In [20]:
#Compute Binary Variable X_ik_ft and Integer Variable Y_ik_ft, Algorithm 1

def GenerateVariablesManutoDist(A_if_t, dis_ik, epsilonk, time_periods, product_types, manufacturers, distributors):
    import numpy as np
    X_ik_ft = np.zeros((manufacturers, distributors))
    Y_ik_ft = np.zeros((manufacturers, distributors))
    IC_if_t = np.zeros((manufacturers, product_types))
    c = 0
    for t in range(1, time_periods):
        for f in range(1, product_types):
            for i in range(1, manufacturers):
                for k in range(1, distributors):
                    if(dis_ik[i][k] < epsilonk[k]):
                        X_ik_ft[i][k] = 1
                        c = c + 1
                IC_if_t[i][f] = c
                c = 0
            
    for f in range(1, product_types):
        for i in range(1, manufacturers):
            for k in range(1, distributors):
                if(X_ik_ft[i][k] == 1):
                    Y_ik_ft[i][k] = np.floor(A_if_t[i][f]/IC_if_t[i][f])
                    
                    
    return (X_ik_ft, Y_ik_ft)

In [21]:
#Compute Binary Variables and Integer Variable X_kl_ft and Integer Variable Y_kl_ft, Algorithm 2.1

def GenerateVariablesDisttoWhole(dis_kl, epsilonk, time_periods, product_types, distributors, wholesalers):
    import numpy as np
    X_kl_ft = np.zeros((distributors, wholesalers))
    Y_kl_ft = np.zeros((distributors, wholesalers))
    KC_kf_t = np.zeros((distributors, product_types))
    c = 0
    
    for t in range(1, time_periods):
        for f in range(1, product_types):
            for k in range(1, distributors):
                for l in range(1, wholesalers):
                    if(dis_ik[k][l] < epsilonk[l]):
                        X_kl_ft[k][l] = 1
                        c = c + 1
                KC_kf_t[k][f] = c
                c = 0
                
        for f in range(1, product_types):
            for k in range(1, distributors):
                for l in range(1, wholesalers):
                    if(X_kl_ft[k][l] == 1 and t == 1):
                        Y_kl_ft[k][l] = ((P_k_ft[k][f] + Z_k_f[k][f])/KC_kf_t[k][f])*np.floor(np.random.randn(1)*100)
                    elif(X_ik_ft[i][k] == 1 and t > 1):
                        Y_kl_ft[k][l] = ((P_k_ft[k][f] + Z_k_f[k][f])/KC_kf_t[k][f])*np.floor(np.random.randn(1)*100)
    
    
    return (X_kl_ft, Y_kl_ft)

In [41]:
#Compute Variables alpha_kl_aft, delta_kl_bft, Z_k_ft, Algorithm 2.2

def ComputeVariables1(dis_kl, epsilonk, phi_f, time_periods, product_types, distributors, wholesalers):
    import numpy as np
    X_kl_ft, Y_kl_ft = GenerateVariablesDisttoWhole(dis_kl, epsilonk, time_periods, product_types, distributors, wholesalers)
    delta_kl_bft = np.zeros((distributors, wholesalers))
    alpha_kl_bft = np.zeros((distributors, wholesalers))
    Z_k_ft = np.zeros(distributors)
    for t in range(1, time_periods):
        for f in range(1, product_types):
            for k in range(1, distributors):
                for l in range(1, wholesalers):
                    delta_kl_bft[k][l] = np.ceil((np.max(0, Y_kl_ft[k][l] - phi_f))/(np.max(0, Y_kl_ft[k][l] - phi_f) + 1))
                    if(X_kl_ft[k][l] == 1 and delta_kl_bft[k][l] == 0):
                        alpha_kl_bft[k][l] = 1
                        
        for f in range(1, product_types):
            for k in range(1, distributors):
                if(t == 1):
                    sum1 = 0
                    for p in range(0, Y_kl_ft.shape[1]):
                        for q in range(0, Y_kl_ft.shape[1]):
                            sum1 = sum1 + Y_kl_ft[p][q]
                        sum1 = 0
                    Z_k_ft[k] = P_k_ft[k] + Z_k_f0[0] - sum1
                elif(t > 1):
                    sum1 = 0
                    for p in range(0, Y_kl_ft.shape[1]):
                        for q in range(0, Y_kl_ft.shape[1]):
                            sum1 = sum1 + Y_kl_ft[p][q]
                        sum1 = 0
                    Z_k_ft[k] = P_k_ft[k] + Z_k_f0[t] - sum1
                else:
                    Z_k_ft[k] = 0  
                    
                    
    return (alpha_kl_aft, delta_kl_bft, Z_k_ft)

In [ ]:
#Compute Variables X_lm_ft, Y_lm_ft, Z_l_ft, Algorithm 3

def ComputeVariables2(Z_l_ft, time_periods, product_types, wholesalers, retailers, epsilonm):
    import numpy as np
    X_lm_ft = np.zeros((wholesalers, retailers))
    Y_lm_ft = np.zeros((wholesalers, retailers))
    LC_lf_t = np.zeros((wholesalers, product_types))
    c = 0
     for t in range(1, time_periods):
        for f in range(1, product_types):
            for l in range(1, wholesalers):
                for m in range(1, retailers):
                    if(dis_lm[l][m] < epsilonm):
                        X_lm_ft[l][m] = 1
                        c = c + 1
                LC_lf_t[l][f] = c
                c = 0
        for f in range(1, product_types):
            for l in range(1, wholesalers):
                for m in range(1, retailers):
                    if(X_lm_ft[l][m] == 1 and t == 1):
                        Y_lm_ft[l][m] = ((P[l] + Z_l_ft[l][0])/LC_lf_t[l][f])*np.floor(np.random.randn(1)*100)
                    elif(X_lm_ft[l][m] == 1 and t > 1):
                        Y_lm_ft[l][m] = ((P[l] + Z_l_ft[l][t])/LC_lf_t[l][f])*np.floor(np.random.randn(1)*100)
                    else:
                        Y_lm_ft[l][m] = 0
                        
                        
        for f in range(1, product_types):
            for l in range(1, wholesalers):
                if(t == 1):
                    